## DUSP1 Spot Detection Visulaization 

This Jupyter Notebook uses **Napari** to visualize pre-generated spot detection results in 3D. It utilizes **Dask** for efficient handling of large image datasets and integrates data from a mounted drive. The workflow includes loading masks and images, processing spot detection results, and rendering them interactively in Napari.


In [ ]:
import napari
import matplotlib.pyplot as plt
import json
import pandas as pd
import h5py 
import numpy as np
import dask.array as da
import os
from tempfile import TemporaryDirectory
import sys

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.Analysis_DUSP1 import DUSP1AnalysisManager

In [ ]:
# Datafrmae location
df = pd.read_csv('/Volumes/share/Users/Eric/DUSP1_May2025/DUSP1_D_Final_FinalSpots_MG3_Abs9_May12_mg_abs.csv')
# H55 file location
h5_dir = '/Volumes/share/Users/Eric/DUSP1_H5/RepD'
h5_files = [
    os.path.join(h5_dir, f)
    for f in os.listdir(h5_dir)
    if f.endswith('.h5') and os.path.isfile(os.path.join(h5_dir, f))
]

In [ ]:
am = DUSP1AnalysisManager(location=h5_files, mac=True) 
am.select_analysis('DUSP1_D_Final')

In [ ]:
img, mask = am.get_images_and_masks()

In [ ]:
for i, (r, m) in enumerate(zip(img, mask)):
    print(f"[{i}] raw_images: {type(r)}, shape={r.shape}, dtype={r.dtype}")
    print(f"[{i}] masks:      {type(m)}, shape={m.shape}, dtype={m.dtype}")

In [ ]:
df.keys()

In [ ]:
# 1) pick your file-index
h5_idx = 1
h5_path = h5_files[h5_idx]

# 2) extract the basename
fn = os.path.basename(h5_path)              
name_noext = fn                              

# 3) filter DataFrame by "endswith" that basename
df_h5 = df[df["NAS_location"].str.endswith(fn)].copy()

# sanity check
print("HDF5 file:  ", h5_path)
print("DF rows:    ", df_h5.shape[0], "points for", fn)

# 4) No need to re-index fovs: your 'fov' is already 0…29
#    Build your points array:
points = np.vstack([
    df_h5["fov"].values,
    df_h5["z_px"].values,
    df_h5["y_px"].values,
    df_h5["x_px"].values,
]).T

# 5) slice out just channel 0 from your raw dask stack
raw5d  = img[h5_idx]          # shape (30,1,3,27,936,640)
mrna4d = raw5d[:, 0, 0]       # now (30, 27, 936, 640)

# 6) launch Napari
viewer = napari.Viewer()
viewer.add_image(
    mrna4d,
    name=name_noext + "_mRNA",
    colormap="gray",
    scale=(5,1,1),
    channel_axis=None
)
viewer.add_points(
    points,
    name=name_noext + "_Spots",
    size=4,
    face_color="red",
    edge_color="red",
    scale=(5,1,1),
    ndim=4
)
napari.run()